# State space model

In [22]:
import torch
from torch import nn 
from torch.distributions.multivariate_normal import MultivariateNormal
from matplotlib import pyplot as plt

z_dim=2
y_dim=2
random_z_mean_sample = torch.rand(z_dim)
random_z_cov_sample = torch.rand(z_dim,z_dim)
random_y_mean_sample = torch.rand(y_dim)
random_y_mean_sample = torch.rand(y_dim,y_dim)

sample_uniform = lambda sample_tensor,low=0,high=1 : torch.distributions.Uniform(low,high).sample(sample_tensor.size())
sample_normal = lambda x : (1/torch.sqrt(torch.tensor([2])*torch.pi))*torch.exp(-0.5*torch.square(x))
pz_init_mean = sample_normal(sample_uniform(random_z_mean_sample))
print(pz_init_mean)

tensor([0.3809, 0.2834])


Why postive definite ?

https://math.stackexchange.com/questions/86309/what-is-the-geometrical-interpretation-to-positive-definite-matrix

In [42]:
from jax.scipy.linalg import cho_factor, cho_solve
import torch
import jax.numpy as jnp
# import numpy as np

def symmetrize(A):
    return 0.5 * (A + torch.swapaxes(A, -1, -2))

def symmetrize_jax(A):
    """Symmetrize one or more matrices."""
    return 0.5 * (A + jnp.swapaxes(A, -1, -2))

def psd_solve_jax(A, b, diagonal_boost=1e-9):
    """A wrapper for coordinating the linalg solvers used in the library for psd matrices."""
    A = symmetrize_jax(A) + diagonal_boost * jnp.eye(A.shape[-1])
    L, lower = cho_factor(A, lower=True)
    print(L)
    x = cho_solve((L, lower), b)
    return x

def psd_solve(A, b, diagonal_boost=1e-9):
    A = symmetrize(A) + diagonal_boost * torch.eye(A.shape[-1])
    L = torch.linalg.cholesky(A)
    print(L)
    x = torch.cholesky_solve(b,L)
    return x

# A_jax = jnp.array([[9, 3, 1, 5], [3, 7, 5, 1], [1, 5, 9, 2], [5, 1, 2, 6]])
# b_jax = jnp.array([1,1,1,1])
# print(psd_solve_jax(A_jax,b_jax))
# A = torch.tensor([[9, 3, 1, 5], [3, 7, 5, 1], [1, 5, 9, 2], [5, 1, 2, 6]])
# b = torch.tensor([[1],[1],[1],[1]])
# print(psd_solve(A,b.float()))

A_jax = jnp.array([[4, -2, 4], [-2, 6, -2], [4, 2, 9]])
b_jax = jnp.array([1,-2,0])
print(psd_solve_jax(A_jax,b_jax))
A = torch.tensor([[4, -2, 4], [-2, 6, -2], [4, 2, 9]],dtype=torch.complex64)
b = torch.tensor([[1],[-2],[0]],dtype=torch.complex64)
print(psd_solve(A,b))

[[ 2.         0.         0.       ]
 [-1.         2.236068   0.       ]
 [ 2.         0.8944272  2.04939  ]]
[ 0.21428575 -0.26190475 -0.09523811]
tensor([[ 2.0000+0.j,  0.0000+0.j,  0.0000+0.j],
        [-1.0000+0.j,  2.2361+0.j,  0.0000+0.j],
        [ 2.0000+0.j,  0.8944+0.j,  2.0494+0.j]])
tensor([[ 0.2143+0.j],
        [-0.2619+0.j],
        [-0.0952+0.j]])
